BÀI 1:


In [13]:
import pandas as pd
import numpy as np

# Dữ liệu mẫu
data = {
    'A': [8, 7.5, 6, 7],
    'B': [9, 8.5, 7, 6]
}

df = pd.DataFrame(data)

# Tính hệ số tương quan Pearson thủ công
n = len(df)
sum_a = df['A'].sum()
sum_b = df['B'].sum()
sum_ab = (df['A'] * df['B']).sum()
sum_a2 = (df['A']**2).sum()
sum_b2 = (df['B']**2).sum()

# Áp dụng công thức
numerator = n * sum_ab - sum_a * sum_b
denominator = np.sqrt((n * sum_a2 - sum_a**2) * (n * sum_b2 - sum_b**2))
r = numerator / denominator

print(f"Hệ số tương quan Pearson giữa A và B: r = {r:.4f}")

# So sánh với hàm có sẵn
print("Hệ số tương quan Pearson (dùng hàm pandas):", df.corr().loc['A', 'B'])


Hệ số tương quan Pearson giữa A và B: r = 0.6911
Hệ số tương quan Pearson (dùng hàm pandas): 0.6910506641398756


BÀI 2:

In [1]:
import pandas as pd
from scipy.stats import chi2_contingency

# Dữ liệu gốc
data = {
    'Day': ['Day 1']*3 + ['Day 2']*3 + ['Day 3']*3 + ['Day 4']*3,
    'Model': ['A', 'B', 'C']*4,
    'Score': [8, 9, 7, 7.5, 8.5, 7, 6, 7, 8, 7, 6, 5]
}

df = pd.DataFrame(data)

# Phân loại điểm số
def categorize(score):
    if score <= 6:
        return 'Low'
    elif score <= 8:
        return 'Medium'
    else:
        return 'High'

df['ScoreCategory'] = df['Score'].apply(categorize)

# Tạo bảng tần số (contingency table)
contingency = pd.crosstab(df['Model'], df['ScoreCategory'])

# In bảng tần số
print("Contingency Table:")
print(contingency)

# Thực hiện kiểm định Chi-squared
chi2, p, dof, expected = chi2_contingency(contingency)

print("\nChi-squared Test Result:")
print("Chi-squared value:", chi2)
print("p-value:", p)
print("Degrees of Freedom:", dof)
print("Expected frequencies:\n", pd.DataFrame(expected, index=contingency.index, columns=contingency.columns))


Contingency Table:
ScoreCategory  High  Low  Medium
Model                           
A                 0    1       3
B                 2    1       1
C                 0    1       3

Chi-squared Test Result:
Chi-squared value: 5.142857142857143
p-value: 0.27295102496702883
Degrees of Freedom: 4
Expected frequencies:
 ScoreCategory      High  Low    Medium
Model                                 
A              0.666667  1.0  2.333333
B              0.666667  1.0  2.333333
C              0.666667  1.0  2.333333


bài 3:


In [15]:
from datetime import datetime

def convert_to_ampm(t):
    hour = t // 100
    minute = t % 100
    dt = datetime.strptime(f"{hour:02d}:{minute:02d}", "%H:%M")
    return dt.strftime("%#I:%M %p")  # Dùng %#I trên Windows để bỏ số 0

# Ví dụ
times = [830, 1445, 30, 0]
converted_times = [convert_to_ampm(t) for t in times]

print(converted_times)


['8:30 AM', '2:45 PM', '12:30 AM', '12:00 AM']


In [18]:
import sqlite3
import pandas as pd

# Kết nối SQLite trong RAM
conn = sqlite3.connect(':memory:')

# Tạo bảng
conn.execute('''
CREATE TABLE students_scores (
    student_id INTEGER,
    name TEXT,
    score FLOAT
);
''')

# Dữ liệu mẫu
data = [
    (1, 'Thành', 9.2), (2, 'An', 9.2), (3, 'Lung', 7.1), (4, 'Dũng', 9.9),
    (5, 'Vũ', 8.8), (6, 'Hương', 9.5), (7, 'Mai Anh', 10.0), (8, 'Hoa', 7.0),
    (9, 'Phong', 6.6), (10, 'Lãm ', 9.2), (11, 'Hải ', 8.5), (12, 'Lanh', 9.8)
]
conn.executemany('INSERT INTO students_scores (student_id, name, score) VALUES (?, ?, ?);', data)
conn.commit()

# Bước 1: lấy số lượng dòng (cnt)
cnt = pd.read_sql_query("SELECT COUNT(*) AS cnt FROM students_scores", conn)['cnt'].iloc[0]

# Bước 2: viết truy vấn có chèn biến cnt
query = f'''
WITH sorted_scores AS (
    SELECT score
    FROM students_scores
    ORDER BY score
),
median_score AS (
    SELECT 
        {"(SELECT score FROM sorted_scores LIMIT 1 OFFSET {cnt//2})" if cnt % 2 == 1 else
         f"(SELECT AVG(score) FROM (SELECT score FROM sorted_scores LIMIT 2 OFFSET {cnt//2 - 1}))"}
        AS median
),
mad_calc AS (
    SELECT 
        ABS(score - (SELECT median FROM median_score)) AS deviation
    FROM students_scores
),
mad_value AS (
    SELECT AVG(deviation) AS mad FROM mad_calc
),
final AS (
    SELECT 
        student_id,
        name,
        score,
        ABS(score - (SELECT median FROM median_score)) AS deviation
    FROM students_scores
)
SELECT 
    student_id,
    name,
    score,
    deviation,
    CASE 
        WHEN deviation > 1.5 * (SELECT mad FROM mad_value) THEN 'Outlier'
        ELSE 'Not Outlier'
    END AS is_outlier
FROM final;
'''

# Chạy truy vấn
outliers_df = pd.read_sql_query(query, conn)
outliers_df


,student_id,name,score,deviation,is_outlier
0,1,Thành,9.2,0.0,Not Outlier
1,2,An,9.2,0.0,Not Outlier
2,3,Lung,7.1,2.1,Outlier
3,4,Dũng,9.9,0.7,Not Outlier
4,5,Vũ,8.8,0.4,Not Outlier
5,6,Hương,9.5,0.3,Not Outlier
6,7,Mai Anh,10.0,0.8,Not Outlier
7,8,Hoa,7.0,2.2,Outlier
8,9,Phong,6.6,2.6,Outlier
9,10,Lãm,9.2,0.0,Not Outlier


In [21]:
import sqlite3
import pandas as pd

# Tạo kết nối SQLite trong RAM
conn = sqlite3.connect(':memory:')

# Tạo bảng Patient
conn.execute('''
CREATE TABLE Patient (
    id INTEGER PRIMARY KEY,
    last_name TEXT,
    weight FLOAT,
    height FLOAT
);
''')

# Thêm dữ liệu mẫu
data = [
    (1, 'Nguyen', 60.0, 165),
    (2, 'Nguyen', 60.5, 166),
    (3, 'Tran', 55.0, 160),
    (4, 'Tran', 70.0, 170),
    (5, 'Nguyen', 80.0, 175)
]
conn.executemany('INSERT INTO Patient (id, last_name, weight, height) VALUES (?, ?, ?, ?);', data)
conn.commit()

# Truy vấn so sánh từng cặp
query = '''
SELECT 
    p1.id AS id1,
    p2.id AS id2,
    p1.last_name AS name1,
    p2.last_name AS name2,
    p1.weight AS weight1,
    p2.weight AS weight2,
    ABS(p1.weight - p2.weight) AS weight_diff,
    LOWER(p1.last_name) = LOWER(p2.last_name) AS same_last_name,
    ABS(p1.weight - p2.weight) <= 1.0 AS similar_weight,
    CASE 
        WHEN LOWER(p1.last_name) = LOWER(p2.last_name) AND ABS(p1.weight - p2.weight) <= 1.0 THEN 'Possibly Same Person'
        ELSE 'Different'
    END AS match_result
FROM Patient p1
JOIN Patient p2 ON p1.id < p2.id
ORDER BY p1.id, p2.id;
'''

# Thực thi truy vấn và hiển thị
result_df = pd.read_sql_query(query, conn)
result_df


,id1,id2,name1,name2,weight1,weight2,weight_diff,same_last_name,similar_weight,match_result
0,1,2,Nguyen,Nguyen,60.0,60.5,0.5,1,1,Possibly Same Person
1,1,3,Nguyen,Tran,60.0,55.0,5.0,0,0,Different
2,1,4,Nguyen,Tran,60.0,70.0,10.0,0,0,Different
3,1,5,Nguyen,Nguyen,60.0,80.0,20.0,1,0,Different
4,2,3,Nguyen,Tran,60.5,55.0,5.5,0,0,Different
5,2,4,Nguyen,Tran,60.5,70.0,9.5,0,0,Different
6,2,5,Nguyen,Nguyen,60.5,80.0,19.5,1,0,Different
7,3,4,Tran,Tran,55.0,70.0,15.0,1,0,Different
8,3,5,Tran,Nguyen,55.0,80.0,25.0,0,0,Different
9,4,5,Tran,Nguyen,70.0,80.0,10.0,0,0,Different
